# Database Preparation

# LGAS Table

In [ ]:
ogr2ogr -lco GEOMETRY_NAME=geom -nln <table_name> Pg:'dbname=<db_name> host=<host_ip> user=<db username> port=<db port> password=<db password>'  lgas_info_last.geojson -append

# GRIDS Table

In [ ]:
ogr2ogr -lco GEOMETRY_NAME=geometry -nln <table_name> Pg:'dbname=<db_name> host=<host_ip> user=<db username> port=<db port> password=<db password>' all_grids_loc.geojson -overwrite / -append

# Image Download Table

In [60]:
import fsspec
import pandas as pd
import os
import glob
import json
import numpy as np

from joblib import Parallel, delayed
from tqdm import tqdm

import shapely.wkt
from dateutil import parser
from datetime import datetime, timezone, timedelta
from tqdm.notebook import tqdm
tqdm.pandas()

In [40]:
fs = fsspec.filesystem("gs", requester_pays = True)

In [41]:
from google.cloud import storage
client = storage.Client()

In [42]:
# List all files available in GCP

#GRID
from tqdm.notebook import tqdm

blobs = client.list_blobs("ml4floods_nema", prefix = "0_DEV/1_Staging/GRID")

files = [x.name for x in tqdm(blobs)]

0it [00:00, ?it/s]

In [43]:
#Categorize into their respective collections


def img_id(row):
    return '_'.join([row['grid'], row['sat'], row['date']])

sat_names = ['Landsat', 'PERMANENTWATERJRC', 'S2']

results = []
for f in tqdm(fnames):
    m = f.split('/')
    if not len(m) < 5:
        m = m[3:]
        if m[1] in sat_names:
            grid = m[0]
            sat = m[1]
            file = m[2]
            results.append(dict(grid= grid, sat = sat, file = file, date = file.split('.')[0], fpath = f))
            
dff = pd.DataFrame(results)

#Split into two dataframes - one for image files, one for CSVs containing cloud/valids metadata.
df_img = dff[dff['file'].str.contains('tif')]
df_info = dff[dff['file'].str.contains('csv')]

#image_id = grid_collection_date . example : GRID05011_S2_2022-10-22
df_img['image_id'] = df_img.apply(img_id, axis=  1)
df_img.columns = ['name', 'satellite', 'file', 'date', 'fpath', 'image_id']
df_img['gcp_filepath'] = "gs://ml4floods_nema/" + df_img['fpath']
images_table = df_img[['image_id', 'name', 'satellite', 'date', 'gcp_filepath']]

  0%|          | 0/76509 [00:00<?, ?it/s]

/var/folders/rz/3bnyyjw92qggqckc99r0_9cc0000gn/T/ipykernel_9873/2970062231.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_img['image_id'] = df_img.apply(img_id, axis=  1)
/var/folders/rz/3bnyyjw92qggqckc99r0_9cc0000gn/T/ipykernel_9873/2970062231.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_img['gcp_filepath'] = "gs://ml4floods_nema/" + df_img['fpath']


In [44]:
# Read all CSVs, and concat into one dataframe

def read_df(i):
    path = "gs://ml4floods_nema/{}".format(df_info.iloc[i]['fpath'])
    temp_df = pd.read_csv(path)
    temp_df['grid'] = df_info.iloc[i]['grid']
    temp_df['sat'] = df_info.iloc[i]['sat']
    temp_df['date'] = temp_df['datetime'].str.split(' ').str[0]
    temp_df['gcs_filepath'] = "https://storage.cloud.google.com/ml4floods_nema/{}".format(df_info.iloc[i]['fpath'])
    return temp_df


result = Parallel(n_jobs=12, backend="threading")(
        delayed(read_df)(i)
        for i in tqdm(range(len(df_info)))
    )


all_info_sat = pd.concat(result)
print(all_info_sat.shape)
all_info_sat.reset_index(inplace = True, drop = True)


#Rename columns
all_info_sat.columns = ['system:time_start', 'valids', 'cloud_probability', 'datetime',
       'index_image_collection', 'name', 'satellite', 'date', 'gcs_filepath']
sat_dedup = all_info_sat.drop_duplicates(subset=['name', 'satellite', 'date'], keep = 'last')

  0%|          | 0/5120 [00:00<?, ?it/s]

(82593, 9)


In [45]:
all_info_sat.shape, sat_dedup.shape

((82593, 9), (81212, 9))

In [58]:
image_download = pd.merge(sat_dedup[['valids', 'cloud_probability','datetime','name', 'satellite', 'date']], df_img[['name', 'satellite', 'date', 'image_id', 'gcp_filepath']],  how = 'outer', on = ['date', 'name', 'satellite'])
invalid_s2_indices = image_download[(image_download['valids'].isna()) & (image_download['satellite'] != 'PERMANENTWATERJRC')].index
image_download = image_download.drop(invalid_s2_indices)
image_download['downloaded'] = image_download['image_id'].apply(lambda x : True if pd.notna(x) else False)

image_download['downloaded'].value_counts()

False    61068
True     22704
Name: downloaded, dtype: int64

In [61]:
#Calculate Solarday and solardatetime

import geopandas as gpd
grids = gpd.read_file('all_grids_loc.geojson')
# grids = gpd.read_csv('grid_loc.csv')
grids_dedup = grids.drop_duplicates(subset = 'name', keep = 'first')

mmdf = pd.merge(image_download, grids_dedup, on = 'name', how = 'left')


def solarday_calc(row):
    if pd.notna(row['geometry']):
        geom = row['geometry']
        # geom = shapely.wkt.loads(row['geometry'])
        longitude = geom.centroid.coords[0][0]
        hours_add = longitude * 12 / 180.
        # if pd.isna(row['datetime']):
            # return row
        if row['satellite'] in ['S2', 'Landsat']:
            dt = parser.parse(row["datetime"])
            row['solardatetime'] = dt + timedelta(hours=hours_add)
            row['solarday'] = row['solardatetime'].strftime("%Y-%m-%d")
            
        return row
    else:
        return row
    
mmdf = mmdf.progress_apply(solarday_calc, axis=1)

  0%|          | 0/83772 [00:00<?, ?it/s]

In [64]:
#Analysis of how many entries dont have matching UTC datetime and solardatetime

def date_solar_match(row):
    if row['solarday']:
        return row['date'] == row['solarday']
    else:
        return np.nan
    
mmdf['date_match'] = mmdf.apply(date_solar_match, axis = 1)
mmdf['date_match'].value_counts(dropna= False)

True     73869
False     9903
Name: date_match, dtype: int64

In [66]:
def apply_image_id(row):
    if pd.isna(row['image_id']):
        if pd.notna(row['name']):
            return "_".join([row['name'], row['satellite'], row['date']])
    return row['image_id']

#Rearranging columns and adding image_id if not present. 

mmdf = mmdf[['image_id', 'name', 'satellite', 'date', 'datetime' ,'gcp_filepath', 'valids', 'cloud_probability', 'solardatetime', 'solarday', 'downloaded']]
mmdf['image_id'] = mmdf.progress_apply(apply_image_id, axis = 1)

  0%|          | 0/83772 [00:00<?, ?it/s]

/var/folders/rz/3bnyyjw92qggqckc99r0_9cc0000gn/T/ipykernel_9873/1339013083.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mmdf['image_id'] = mmdf.progress_apply(apply_image_id, axis = 1)


In [67]:
def change_date(row):
    if row['satellite'] != 'PERMANENTWATERJRC' and row['date'] != row['solarday']:
        if pd.notna(row['gcp_filepath']):
            row['to_delete'] = True
            row['date'] = row['solarday']
            row['to_delete_url'] = row['gcp_filepath']
            row['gcp_filepath'] = None
            row['downloaded'] = False
            row['image_id'] = "_".join([row['name'], row['satellite'], row['date']])
        else:
            row['date'] = row['solarday']
            row['image_id'] = "_".join([row['name'], row['satellite'], row['date']])
    return row

#Changing dates for those where UTC and solardatetime dont match. 
mmdf = mmdf.progress_apply(change_date, axis = 1)

  0%|          | 0/83772 [00:00<?, ?it/s]

In [75]:
image_download_table = mmdf[['image_id', 'name', 'satellite', 'date', 'datetime', 'downloaded', 'gcp_filepath', 'cloud_probability', 'valids', 'solardatetime', 'solarday']]
image_download_table['in_progress'] = 0

/var/folders/rz/3bnyyjw92qggqckc99r0_9cc0000gn/T/ipykernel_9873/21118546.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  image_download_table['in_progress'] = 0


In [76]:
image_download_table.head()

,image_id,name,satellite,date,datetime,downloaded,gcp_filepath,cloud_probability,valids,solardatetime,solarday,in_progress
0,GRID02947_Landsat_2022-08-21,GRID02947,Landsat,2022-08-21,2022-08-21 00:22:44.081500+00:00,False,NaN,0.001414,1.000000,2022-08-21 09:46:36.471340+00:00,2022-08-21,0
1,GRID02947_Landsat_2022-09-06,GRID02947,Landsat,2022-09-06,2022-09-06 00:22:45.466500+00:00,False,NaN,0.948793,1.000000,2022-09-06 09:46:37.856340+00:00,2022-09-06,0
2,GRID02947_Landsat_2022-10-08,GRID02947,Landsat,2022-10-08,2022-10-08 00:22:46.536500+00:00,False,NaN,0.371418,1.000000,2022-10-08 09:46:38.926340+00:00,2022-10-08,0
3,GRID02947_Landsat_2022-08-05,GRID02947,Landsat,2022-08-05,2022-08-05 00:22:51.440000+00:00,False,NaN,0.020780,0.352657,2022-08-05 09:46:43.829840+00:00,2022-08-05,0
4,GRID02947_Landsat_2022-08-13,GRID02947,Landsat,2022-08-13,2022-08-13 00:22:22.858000+00:00,False,NaN,0.065566,1.000000,2022-08-13 09:46:15.247840+00:00,2022-08-13,0


# Model Inference Table

In [1]:
import fsspec
import pandas as pd
import glob
from google.cloud import storage
from collections import defaultdict

In [2]:
fs = fsspec.filesystem("gs", requester_pays = True)
client = storage.Client()

In [3]:
#GRID
from tqdm.notebook import tqdm

blobs = client.list_blobs("ml4floods_nema", prefix = "0_DEV/1_Staging/GRID")

files = [x.name for x in tqdm(blobs)]

0it [00:00, ?it/s]

In [10]:
sat_names = ['WF2_unet_rbgiswirs', 'WF2_unet_rbgiswirs_cont', 'WF2_unet_rbgiswirs_vec']

results = defaultdict(dict)
for f in tqdm(files):
    m = f.split('/')
    if not len(m) < 5:
        m = m[3:]
        if m[1] in sat_names:
            name = m[0]
            inf_type= m[1]
            satellite = m[2]
            date = m[3].split('.')[0]
            file = m[3]
            gcs_filepath = "gs://ml4floods_nema/{}".format(f)
            image_id = "_".join([name, satellite, date])
            
            results[image_id][inf_type] = gcs_filepath
            results[image_id].update(dict(image_id = image_id, name = name, satellite = satellite, date = date, model_id = "WF2_unet_rbgiswirs"))
            
            
df = pd.DataFrame(results.values())
df = df[['image_id', 'name', 'satellite', 'date', 'model_id', 'WF2_unet_rbgiswirs', 'WF2_unet_rbgiswirs_cont', 'WF2_unet_rbgiswirs_vec']]
df.rename(columns={'WF2_unet_rbgiswirs' : 'prediction', 'WF2_unet_rbgiswirs_cont' : 'prediction_cont', 'WF2_unet_rbgiswirs_vec' : 'prediction_vec'}, inplace = True)

dff['session_data'] = '{}'

  0%|          | 0/76509 [00:00<?, ?it/s]

# Postprocessing Temporal

In [11]:
import fsspec
import pandas as pd
import glob
from google.cloud import storage
import geopandas as gpd
import numpy as np
from collections import defaultdict

In [12]:
fs = fsspec.filesystem("gs", requester_pays = True)
client = storage.Client()

In [23]:
s2folder_file = "gs://ml4floods_nema/0_DEV/1_Staging/operational/*/*/pre_post_products/*.geojson"
s2files = fs.glob(f"{s2folder_file}")
# s2files = [f"gs://{s2}" for s2 in s2files]

In [24]:
rows = []

#Get all existing postprocessed geojsons from GCP, and split the files into pre, post, and prepost flood events, for every AOI. 

for file in s2files:
    try:
        if 'NEMA001' in file:
            continue
        bucket, _, _, _, session_code, name, _, fname = file.split('/')
        flooding_date_pre_end_str, flooding_date_post_start_str, flooding_date_post_end_str = None, None, None
        if 'prepostflood_' in fname:
            flooding_date_pre_end_str = fname.split('_')[1]
            flooding_date_post_start_str = fname.split('_')[2]
            flooding_date_post_end_str = fname.split('_')[3].split('.')[0]
            t = 'prepostflood'
        elif 'preflood_' in fname:
            flooding_date_pre_end_str = fname.split('_')[1].split('.')[0]
            t = 'preflood'
        elif 'postflood_' in fname:
            flooding_date_post_start_str = fname.split('_')[1]
            flooding_date_post_end_str = fname.split('_')[2].split('.')[0]
            t = 'postflood'
        rows.append({"bucket" : bucket, "session" : session_code, "name" : name, "fname" : fname, 
                    "flooding_date_pre_end" : flooding_date_pre_end_str, "flooding_date_post_start" : flooding_date_post_start_str, 
                     "flooding_date_post_end" : flooding_date_post_end_str,
                     "gs_fname" : f"gs://{file}",
                    'type' : t})
    except Exception as e:
        print(file, e)
        
        
df = pd.DataFrame(rows)

In [27]:
new_rows = []

df_merged = df.copy()

grouped_df = df.groupby(['name', 'session'])

for name, session_df in grouped_df:

    prepostflood_rows = session_df[session_df['type'] == 'prepostflood']

    for i, prepostflood_row in prepostflood_rows.iterrows():
        preflood_row = session_df[(session_df['type'] == 'preflood') & (session_df['flooding_date_pre_end'] == prepostflood_row['flooding_date_pre_end'])].iloc[0]

        postflood_row = session_df[(session_df['type'] == 'postflood') & (session_df['flooding_date_post_start'] == prepostflood_row['flooding_date_post_start']) & (session_df['flooding_date_post_end'] == prepostflood_row['flooding_date_post_end'])].iloc[0]

        new_row = {
            'name': name[0],
            'session': name[1],
            'bucket': prepostflood_row['bucket'],
            'model_name' : 'WF2_unet_rbgiswirs',
            # 'preflood_fname': preflood_row['fname'],
            # 'postflood_fname': postflood_row['fname'],
            'preflood': preflood_row['gs_fname'],
            'postflood': postflood_row['gs_fname'],
            'prepostflood' : prepostflood_row['gs_fname'],
            'flooding_date_pre_start': np.nan,
            'flooding_date_pre_end': preflood_row['flooding_date_pre_end'],
            'flooding_date_post_start': postflood_row['flooding_date_post_start'],
            'flooding_date_post_end': postflood_row['flooding_date_post_end']
        }
        new_rows.append(new_row)
        
df = pd.DataFrame(new_rows)

# Postprocessing Spatial

In [ ]:
# s2folder_file = "gs://ml4floods_nema/0_DEV/1_Staging/operational/NEMA002/*"

# grid_names = [x.split('/')[-1] for x in fs.glob(s2folder_file) if "GRID" in x.split('/')[-1]]